##### Copyright 2018 The TF-Agents Authors.

# Introduction

強化学習（Reinforcement learning, RL）はエージェントが環境内で報酬を最大化するためのアクションを学習する一般的なフレームワークです。 2つの主な要素は、解決すべき問題を表す環境と、学習アルゴリズムを表すエージェントです。

エージェントと環境は継続的に相互に作用します。各時間ステップで、エージェントはその*方策（policy）* $\pi(a_t|s_t)$に基づいて環境に対してアクションをとります。ここで$s_t$は現在の環境の観測値であり、アクションをとることで報酬$r_{t+1}$とその次の環境の観測値$s_{t+1}$を得ます。目標は、報酬の合計（リターン）を最大化するように方策を改善することです。

注：環境の「`状態`」とエージェントが見ることができる環境の「`状態`」の一部である「`観察`」を区別することが重要です。例えばポーカーゲームでは環境の状態はすべてのプレイヤーの手札とコミュニティカードで構成されますが、エージェントは自分の手札と限られたコミュニティカードしか観察できません。ほとんどの文献ではこれらの用語は同じ意味で使用されており観測値は$s$とも呼ばれます。

---

Reinforcement learning (RL) is a general framework where agents learn to perform actions in an environment so as to maximize a reward. The two main components are the environment, which represents the problem to be solved, and the agent, which represents the learning algorithm.

The agent and environment continuously interact with each other. At each time step, the agent takes an action on the environment based on its *policy* $\pi(a_t|s_t)$, where $s_t$ is the current observation from the environment, and receives a reward $r_{t+1}$ and the next observation $s_{t+1}$ from the environment. The goal is to improve the policy so as to maximize the sum of rewards (return).

Note: It is important to distinguish between the `state` of the environment and the `observation`, which is the part of the environment `state` that the agent can see, e.g. in a poker game, the environment state consists of the cards belonging to all the players and the community cards, but the agent can observe only its own cards and a few community cards. In most literature, these terms are used interchangeably and observation is also denoted as $s$.

![Agent-Environment Interation Loop](images/rl_overview.png)

これは非常に一般的なフレームワークであり、ゲーム、ロボットなどのようなさまざまな連続的な意思決定問題をモデル化することができます。 

---

This is a very general framework and can model a variety of sequential decision making problems such as games, robotics etc. 

# The Cartpole Environment

Cartpoleは最もよく知られている古典的な強化学習の問題の1つです（RLにおける*"Hello, World!"*）。ポールはカートに取り付けられており、カートは摩擦のないトラックに沿って移動できます。ポールは直立している状態から始まり、目標はカートを制御してポールが倒れないようにすることです。

*   環境の観測値$s_t$はカートの位置と速度、そしてポールの角度と角速度から成る4Dベクトルです。
*   エージェントは、2つのアクション$a_t$のうちの1つを取ることができます。：カートを右（+1）または左（-1）に押す。
*   支柱が直立している状態が確認できるたびに報酬$r_{t+1} = 1$が与えられます。次のいずれかに該当する場合、エピソードは終了します。 
  * ポールの傾きが限界の角度を超える。
  * カートの位置が横に移動しすぎたために限界を超える。
  * 200時間ステップが経過する。

エージェントの目的は、エピソード$\sum_{t=0}^{T} \gamma^t r_t$で報酬の合計を最大にするような方策$\pi(a_t|s_t)$を学ぶことです。ここで$\gamma$は$[0, 1]$の割引係数であり、即時報酬と比較して将来の報酬を割引きます。このパラメータはより近い時間ステップ後に得られる報酬を重視するように方策が学習するのを助けます。

---

The Cartpole environment is one of the most well known classic reinforcement learning problems ( the *"Hello, World!"* of RL). A pole is attached to a cart, which can move along a frictionless track. The pole starts upright and the goal is to prevent it from falling over by controlling the cart.  

*   The observation from the environment $s_t$ is a 4D vector representing the position and velocity of the cart, and the angle and angular velocity of the pole. 
*   The agent can control the system by taking one of 2 actions $a_t$: push the cart right (+1) or left (-1). 
*   A reward $r_{t+1} = 1$ is provided for every timestep that the pole remains upright. The episode ends when one of the following is true:
  * the pole tips over some angle limit
  * the cart moves outside of the world edges
  * 200 time steps pass. 

The goal of the agent is to learn a policy $\pi(a_t|s_t)$ so as to maximize the sum of rewards in an episode $\sum_{t=0}^{T} \gamma^t r_t$. Here $\gamma$ is a discount factor in $[0, 1]$ that discounts future rewards relative to immediate rewards. This parameter helps us focus the policy, making it care more about obtaining rewards quickly.

# The DQN Agent

 [DQN (Deep Q-Network) algorithm](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) は2015年にDeepMindによって開発されたアルゴリズムです。このアルゴリズムは強化学習とディープニューラルネットワークを組み合わせることで、幅広いAtariゲームを解くことができました。（一部は人間のレベルを超えています。）このアルゴリズムは、Q-Learningと呼ばれる古典的なRLアルゴリズムをディープニューラルネットワークと、*experience replay*と呼ばれる技術によって強化することによって開発されました。

---

The  [DQN (Deep Q-Network) algorithm](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) was developed by DeepMind in 2015. It was able to solve a wide range of Atari games (some to superhuman level) by combining reinforcement learning and deep neural networks at scale. The algorithm was developed by enhancing a classic RL algorithm called Q-Learning with deep neural networks and a technique called *experience replay*.

## Q-Learning

Q-LearningはQ関数の概念に基づいています。方策$\pi$のQ関数$Q^{\pi}(s, a)$（別名、state-action value関数）は、最初に$a$を実行し、その後方策$\pi$に従うことによって状態$s$から得られる期待収益または割引総和を測定します。観測$s$から始めて行動$a$を取り、その後最適方針に従うことで得られる最大収益として、最適なQ関数$Q^*(s, a)$を定義します。最適なQ関数は、次の*Bellman*最適化方程式に従います。

$\begin{equation}
Q^*(s, a) = \mathbb{E}\left[ r + \gamma \max_{a'} Q^*(s', a')\right]
\end{equation}$

これは、状態$s$および行動$a$からの最大収益が、即時報酬$r$と、エピソードの終わりまでの最適な方針に従って得られた（$\gamma$で割り引かれた）収益（つまり、次の状態$s'$からの最大報酬）の合計であることを意味します。 期待値は即時報酬$r$と可能な次の状態$s'$の両方の配分によって計算されます。

Q-Learningの背景にある基本的な考え方は、Bellman最適化方程式を反復計算によって更新することです。 $Q_{i+1}(s, a) \leftarrow \mathbb{E}\left[ r + \gamma \max_{a'} Q_{i}(s', a')\right]$, そしてこれが最適な$Q$-function, i.e. $Q_i \rightarrow Q^*$ as $i \rightarrow \infty$に収束することが分かります。  (see the [DQN paper](https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf)).

---

Q-Learning is based on the notion of a Q-function. The Q-function (a.k.a the state-action value function) of a policy $\pi$, $Q^{\pi}(s, a)$, measures the expected return or discounted sum of rewards obtained from state $s$ by taking action $a$ first and following policy $\pi$ thereafter. We define the optimal Q-function $Q^*(s, a)$ as the maximum return that can be obtained starting from observation $s$, taking action $a$ and following the optimal policy thereafter. The optimal Q-function obeys the following *Bellman* optimality equation: 

$\begin{equation}
Q^*(s, a) = \mathbb{E}\left[ r + \gamma \max_{a'} Q^*(s', a')\right]
\end{equation}$

This means that the maximum return from state $s$ and action $a$ is the sum of the immediate reward $r$ and the return (discounted by $\gamma$) obtained by following the optimal policy therafter until the end of the episode (i.e., the maximum reward from the next state $s'$). The expectation is computed both over the distribution of immediate rewards $r$ and possible next states $s'$.

The basic idea behind Q-Learning is to use the Bellman optimality equation as an iterative update $Q_{i+1}(s, a) \leftarrow \mathbb{E}\left[ r + \gamma \max_{a'} Q_{i}(s', a')\right]$, and it can be shown that this converges to the optimal $Q$-function, i.e. $Q_i \rightarrow Q^*$ as $i \rightarrow \infty$ (see the [DQN paper](https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf)).

## Deep Q-Learning

ほとんどの問題において、$s$と$a$の各組み合わせの値をテーブルとして$Q$関数を表現するのは実用的ではありません。
代わりにQ関数$Q(s, a; \theta) \approx Q^*(s, a)$を表現するためにパラメータ$\theta$を持つニューラルネットワークのような関数近似器を学習します。これは各ステップ$i$で次のような損失を最小限にすることで実現できます。

$\begin{equation}L_i(\theta_i) = \mathbb{E}_{s, a, r, s'\sim \rho(.)} \left[ (y_i - Q(s, a; \theta_i))^2 \right]\end{equation}$ where $y_i = r +  \gamma \max_{a'} Q(s', a'; \theta_{i-1})$

ここで$y_i$はTD (temporal difference)ターゲットと呼ばれます。そして$y_i - Q$はTD誤差と呼ばれます。$ \ rho $は、挙動分布、つまり環境から収集された遷移$\{s, a, r, s'\}$の分布を表します。

前回の反復からのパラメータ$\theta_{i-1}$は固定されていて更新されないことに注意してください。実際には、最新の反復ではなく数回前の反復からのネットワークパラメータのスナップショットを使用します。このコピーは*ターゲットネットワーク*と呼ばれます。

Q-Learningは、方策$a = \max_{a} Q(s, a; \theta)$について学習する*off-policy*のアルゴリズムで、環境内での行動やデータの収集には異なる振る舞い方策を使用します。この振る舞い方策は状態アクション空間をカバーするために、確率$1-\epsilon$で方策に基づく行動を選択し、確率$\epsilon$でランダムに行動を選択する$\epsilon$-greedy方策です。

---

For most problems, it is impractical to represent the $Q$-function as a table containing values for each combination of $s$ and $a$. Instead, we train a function approximator, such as a neural network with parameters $\theta$, to estimate the Q-values, i.e. $Q(s, a; \theta) \approx Q^*(s, a)$. This can done by minimizing the following loss at each step $i$:

$\begin{equation}L_i(\theta_i) = \mathbb{E}_{s, a, r, s'\sim \rho(.)} \left[ (y_i - Q(s, a; \theta_i))^2 \right]\end{equation}$ where $y_i = r +  \gamma \max_{a'} Q(s', a'; \theta_{i-1})$

Here, $y_i$ is called the TD (temporal difference) target, and $y_i - Q$ is called the TD error. $\rho$ represents the behaviour distribution, the distribution over transitions $\{s, a, r, s'\}$ collected from the environment. 

Note that the parameters from the previous iteration $\theta_{i-1}$ are fixed and not updated. In practice we use a snapshot of the network parameters from a few iterations ago instead of the last iteration. This copy is called the *target network*.

Q-Learning is an *off-policy* algorithm that learns about the greedy policy $a = \max_{a} Q(s, a; \theta)$ while using a different behaviour policy for acting in the environment/collecting data. This behaviour policy is usually an $\epsilon$-greedy policy that selects the greedy action with probability $1-\epsilon$ and a random action with probability $\epsilon$ to ensure good coverage of the state-action space.

## Experience Replay

DQN lossにおける完全な期待値の計算を避けるために、確率的勾配降下法を使用してそれを最小化することができます。最後の遷移$\{s, a, r, s'\}$だけを使って損失が計算されると、これは通常のQ-Learningになります。 

Atari DQNでは、ネットワークの更新をより安定させるためにExperience Replayという手法が導入されました。データ収集の各タイムステップで、遷移は*replay buffer*と呼ばれる循環バッファに追加されます。次に、トレーニング中に、損失とその勾配を計算するために最新の遷移だけを使用するのではなく、再生バッファからサンプリングされた遷移のミニバッチを使用してそれらを計算します。これには2つの利点があります。多くの更新で各遷移を再利用することによるデータ効率の向上と、相関のない遷移をまとめて使用することによる学習の安定性の向上です。

---

To avoid computing the full expectation in the DQN loss, we can minimize it using stochastic gradient descent. If the loss is computed using just the last transition $\{s, a, r, s'\}$, this reduces to standard Q-Learning. 

The Atari DQN work introduced a technique called Experience Replay to make the network updates more stable. At each time step of data collection, the transitions are added to a circular buffer called the *replay buffer*. Then during training, instead of using just the latest transition to compute the loss and its gradient, we compute them using a mini-batch of transitions sampled from the replay buffer. This has two advantages: better data efficiency by reusing each transition in many updates, and better stability using uncorrelated transitions in a batch.

# DQN on Cartpole in TF-Agents


TFエージェントはDQNエージェントをトレーニングするために必要なすべてのコンポーネントを提供します。（エージェント、環境、ポリシー、ネットワーク、再生バッファ、データ収集ループ、メトリックなど）これらのコンポーネントはPython関数またはTensorFlowグラフ操作として実装されており、それらの間で変換するためのラッパーもあります。さらに、TF-AgentsはTensorFlow 2.0モードをサポートしているため、TFを命令モードで使用できます。

---

TF-Agents provides all the components necessary to train a DQN agent, such as the agent itself, the environment, policies, networks, replay buffers, data collection loops, and metrics. These components are implemented as Python functions or TensorFlow graph ops, and we also have wrappers for converting between them. Additionally, TF-Agents supports TensorFlow 2.0 mode, which enables us to use TF in imperative mode. 

Next, take a look at the [tutorial for training a DQN agent on the Carpole environment using TF-Agents](https://github.com/tensorflow/agents/blob/master/tf_agents/colabs/1_dqn_tutorial.ipynb).

